In [1]:
# make sure credentials are set
# !export =
#doing that later

In [2]:
# # install particular dependencies (just for reference here)
# !pip install git+https://github.com/pangeo-forge/pangeo-forge-runner.git #pangeo-forge-runner from main

In [3]:
# global input args
repo = "https://github.com/jbusecke/cmip6-leap-feedstock.git"
ref = "beam-dataflow-testing"
f_name = "config.json"
target_bucket = "gs://leap-persistent/jbusecke/data/CMIP6/dataflow_test/target"

In [4]:
def shorten_id(iid):
    iid_stripped = ''.join(iid.lower().split('.')[3:])
    assert len(iid_stripped)<66
    return iid_stripped

In [5]:
# from the integration tests of pangeo-forge-runner
#https://github.com/pangeo-forge/pangeo-forge-runner/blob/main/tests/integration/test_dataflow_integration.py

import subprocess
import os
import time
import xarray as xr

my_env = os.environ.copy()
my_env["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/juliusbusecke/Downloads/keys/leap-pangeo-dataflow_service_account.json"

iid = "CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r107i1p1f1.day.psl.gr.v20200412"
jobname=shorten_id(iid)
target_store = f"{target_bucket}/{iid}.zarr"
machine_type = "e2-highmem-8"
cmd = [
    "pangeo-forge-runner",
    "bake",
    "--repo",
    repo,
    "--ref",
    ref,
    "--json",
    "-f",
    f_name,
    "--Bake.recipe_id",
    iid,
    "--Bake.job_name",
    jobname,
    "--TargetStorage.root_path",
    f"{target_bucket}",
    "--DataflowBakery.machine_type",
    machine_type,
]

print("\nSubmitting job...")
submit_proc = subprocess.run(cmd, capture_output=True, env=my_env)
assert submit_proc.returncode == 0
lastline = json.loads(submit_proc.stdout.decode().splitlines()[-1])
assert lastline["status"] == "submitted"
job_id = lastline["job_id"]
job_name = lastline["job_name"]
print(f"Job submitted with {job_id = }")


Submitting job...


AssertionError: 

In [12]:
a = submit_proc.stdout#.json.flush()

In [15]:
a.splitlines()

[b'{"message": "Target Storage is FSSpecTarget(GCSFileSystem(, root_path=\\"gs://leap-persistent/jbusecke/data/CMIP6/dataflow_test/target\\")\\n", "status": "setup"}',
 b'{"message": "Input Cache Storage is CacheFSSpecTarget(GCSFileSystem(, root_path=\\"gs://leap-scratch/jbusecke/data/CMIP6/dataflow_test/input-cache/\\")\\n", "status": "setup"}',
 b'{"message": "Metadata Cache Storage is MetadataTarget(AbstractFileSystem(, root_path=\\"\\")\\n", "status": "setup"}',
 b'{"message": "Picked Git content provider.\\n", "status": "fetching"}',
 b'{"message": "Cloning into \'/var/folders/_1/1k9jtjl51z333f21s7yht0340000gn/T/tmp8f_cvwjd\'...\\n", "status": "fetching"}',
 b'{"message": "HEAD is now at 033a8dd Merge branch \'beam-dataflow-testing\' of github.com:jbusecke/CMIP6-LEAP-feedstock into beam-dataflow-testing\\n", "status": "fetching"}',
 b'{"message": "Parsing recipes...", "status": "running"}',
 b'{"message": "Error during running: \'version\'", "exc_info": "Traceback (most recent cal